

This uses the BCI Comp III, Dataset # 2 from a P300 speller http://www.bbci.de/competition/iii/desc_II.pdf

The winning accuracy was 96.5% for sampling of 15 trials and 73.5% for 5 trials http://www.bbci.de/competition/iii/results/#winners


Continuing on example found at https://github.com/venthur/wyrm/blob/master/examples/



In [1]:
from __future__ import division
%matplotlib inline
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib as mpl
import pandas as pd

from wyrm import plot
plot.beautify()
from wyrm.types import Data
from wyrm import processing as proc
from wyrm.io import load_bcicomp3_ds2

/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
TRAIN_A = 'data/BCI_Comp_III_Wads_2004/Subject_A_Train.mat'
TRAIN_B = 'data/BCI_Comp_III_Wads_2004/Subject_B_Train.mat'

TEST_A = 'data/BCI_Comp_III_Wads_2004/Subject_A_Test.mat'
TEST_B = 'data/BCI_Comp_III_Wads_2004/Subject_B_Test.mat'

TRUE_LABELS_A = 'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU'
TRUE_LABELS_B = 'MERMIROOMUHJPXJOHUVLEORZP3GLOO7AUFDKEFTWEOOALZOP9ROCGZET1Y19EWX65QUYU7NAK_4YCJDVDNGQXODBEV2B5EFDIDNR'

MATRIX = ['abcdef',
          'ghijkl',
          'mnopqr',
          'stuvwx',
          'yz1234',
          '56789_']

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {'flashing': ['flashing']}

SEG_IVAL = [0, 700]

JUMPING_MEANS_IVALS_A = [150, 220], [200, 260], [310, 360], [550, 660] # 91%
JUMPING_MEANS_IVALS_B = [150, 250], [200, 280], [280, 380], [480, 610] # 91%

In [3]:

def preprocessing_simple(dat, MRK_DEF, *args, **kwargs):
    """Simple preprocessing that reaches 97% accuracy.
    """
    fs_n = dat.fs / 2
    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    dat = proc.filtfilt(dat, b, a)
   
    dat = proc.subsample(dat, 20)
    epo = proc.segment_dat(dat, MRK_DEF, SEG_IVAL)
    fv = proc.create_feature_vectors(epo)
    return fv, epo

In [4]:
def preprocessing(dat, MRK_DEF, JUMPING_MEANS_IVALS):
    dat = proc.sort_channels(dat)
    
    fs_n = dat.fs / 2
    b, a = proc.signal.butter(5, [30 / fs_n], btype='low')
    dat = proc.lfilter(dat, b, a)
    b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
    dat = proc.lfilter(dat, b, a)
    
    dat = proc.subsample(dat, 60)
    epo = proc.segment_dat(dat, MRK_DEF, SEG_IVAL)
    
    fv = proc.jumping_means(epo, JUMPING_MEANS_IVALS)
    fv = proc.create_feature_vectors(fv)
    return fv, epo

### Target accuracy using LDA

In [5]:
epo = [None, None]
acc = 0
for subject in range(2):
    if subject == 0:
        training_set = TRAIN_A
        testing_set = TEST_A
        labels = TRUE_LABELS_A
        jumping_means_ivals = JUMPING_MEANS_IVALS_A
    else:
        training_set = TRAIN_B
        testing_set = TEST_B
        labels = TRUE_LABELS_B
        jumping_means_ivals = JUMPING_MEANS_IVALS_B
    
    # load the training set
    dat = load_bcicomp3_ds2(training_set)
    fv_train, epo[subject] = preprocessing(dat, MARKER_DEF_TRAIN, jumping_means_ivals)
    
    # train the lda
    cfy = proc.lda_train(fv_train)
    
    # load the testing set
    dat = load_bcicomp3_ds2(testing_set)
    fv_test, _ = preprocessing(dat, MARKER_DEF_TEST, jumping_means_ivals)
    
    # predict
    lda_out_prob = proc.lda_apply(fv_test, cfy)
    
    # unscramble the order of stimuli
    unscramble_idx = fv_test.stimulus_code.reshape(100, 15, 12).argsort()
    static_idx = np.indices(unscramble_idx.shape)
    lda_out_prob = lda_out_prob.reshape(100, 15, 12)
    lda_out_prob = lda_out_prob[static_idx[0], static_idx[1], unscramble_idx]
    
    #lda_out_prob = lda_out_prob[:, :5, :]
    
    # destil the result of the 15 runs
    #lda_out_prob = lda_out_prob.prod(axis=1)
    lda_out_prob = lda_out_prob.sum(axis=1)
        
    # 
    lda_out_prob = lda_out_prob.argsort()
    
    cols = lda_out_prob[lda_out_prob <= 5].reshape(100, -1)
    rows = lda_out_prob[lda_out_prob > 5].reshape(100, -1)
    text = ''
    for i in range(100):
        row = rows[i][-1]-6
        col = cols[i][-1]
        letter = MATRIX[row][col]
        text += letter
    print
    print 'Result for subject %d' % (subject+1)
    print 'Constructed labels: %s' % text.upper()
    print 'True labels       : %s' % labels
    a = np.array(list(text.upper()))
    b = np.array(list(labels))
    accuracy = np.count_nonzero(a == b) / len(a)
    print 'Accuracy: %.1f%%' % (accuracy * 100)
    acc += accuracy
print
print 'Overal accuracy: %.1f%%' % (100 * acc / 2)


Result for subject 1
Constructed labels: WQXPLZCOMRKOW7YFZDEZ1DPI9NN2GRKDJCUJRMEUOCOJD2UFYPOO6J7LDGYEGOA5VHNEKBW4OO1TDOILUEE5BFAEEXAW_K3R3MRU
True labels       : WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
Accuracy: 91.0%

Result for subject 2
Constructed labels: MERMIROOMUZJPXJOHUVFBORZP3GLOO7AUFDKEFTWEOOALZOP9R1CGZE11Y19EWX65QUYU7NAK_1ACJDVDNGQXOJBEV2B5EFDIDTR
True labels       : MERMIROOMUHJPXJOHUVLEORZP3GLOO7AUFDKEFTWEOOALZOP9ROCGZET1Y19EWX65QUYU7NAK_4YCJDVDNGQXODBEV2B5EFDIDNR
Accuracy: 91.0%

Overal accuracy: 91.0%


# Using a Multi Layer Perceptron Network

In [6]:
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LearningRateScheduler
from sklearn.pipeline import Pipeline
from keras.constraints import maxnorm


Using TensorFlow backend.


## Network Description

Looks like a shallow network topology so far gets the best results of around 84%. Which is not bad and would have been close to the top 5 results in the original competition. As a next step I will use cross validation and see how the accuracy and fit diverge on the training and validation portions to see if perhaps more epochs are needed to train further.

Network topology:
[256 inputs with 20% dropout] -->  [64 neuron hidden rectifier layer with 30% dropout] --> [1 output sigmoid neuron]


In [23]:
# create model
def create_model():
  
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(256,)))
    model.add(Dense(64, init='normal', activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(0.3))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    
    sgd = SGD(lr=0.1, momentum=0.9, decay=0.001, nesterov=False)
    
    # Compile model
    
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy']) 
    return model


In [24]:

# learning rate schedule
def step_decay(epoch):
    initial_lrate = .9
    drop = 0.5
    epochs_drop = 1
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# learning schedule callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [27]:
#build pipeline with standartization 

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=500,
    batch_size=30, verbose=0)))
pipeline = Pipeline(estimators)

In [28]:
epo = [None, None]
acc = 0
for subject in range(2):
    if subject == 0:
        training_set = TRAIN_A
        testing_set = TEST_A
        labels = TRUE_LABELS_A
        jumping_means_ivals = JUMPING_MEANS_IVALS_A
    else:
        training_set = TRAIN_B
        testing_set = TEST_B
        labels = TRUE_LABELS_B
        jumping_means_ivals = JUMPING_MEANS_IVALS_B
    
    # load the training set
    dat = load_bcicomp3_ds2(training_set)
    fv_train, epo[subject] = preprocessing(dat, MARKER_DEF_TRAIN, jumping_means_ivals)
    
    # train the MLP
    pipeline.fit(fv_train.data, fv_train.axes[0])
    
    # load the testing set
    dat = load_bcicomp3_ds2(testing_set)
    fv_test, _ = preprocessing(dat, MARKER_DEF_TEST, jumping_means_ivals)
    
    # predict
    mlp_out_prob = pipeline.predict_proba(fv_test.data)
    
    # unscramble the order of stimuli
    unscramble_idx = fv_test.stimulus_code.reshape(100, 15, 12).argsort()
    static_idx = np.indices(unscramble_idx.shape)
    mlp_out_prob = mlp_out_prob[:,1].reshape(100, 15, 12)
    mlp_out_prob = mlp_out_prob[static_idx[0], static_idx[1], unscramble_idx]
    
    
    
    # destil the result of the 15 runs
    
    mlp_out_prob = mlp_out_prob.sum(axis=1)
        
    # 
    mlp_out_prob = mlp_out_prob.argsort()
    
    cols = mlp_out_prob[mlp_out_prob <= 5].reshape(100, -1)
    rows = mlp_out_prob[mlp_out_prob > 5].reshape(100, -1)
    text = ''
    for i in range(100):
        row = rows[i][-1]-6
        col = cols[i][-1]
        letter = MATRIX[row][col]
        text += letter
    print
    print 'Result for subject %d' % (subject+1)
    print 'Constructed labels: %s' % text.upper()
    print 'True labels       : %s' % labels
    a = np.array(list(text.upper()))
    b = np.array(list(labels))
    accuracy = np.count_nonzero(a == b) / len(a)
    print 'Accuracy: %.1f%%' % (accuracy * 100)
    acc += accuracy
print
print 'Overal accuracy: %.1f%%' % (100 * acc / 2)


Result for subject 1
Constructed labels: WQXDLZCOMRKUK7YFYDEZ1DQJ9NN2GR_DKUUJRMEUCCOJD2UFYPOOKJ7LDGYEGOA5VXNEHBU4OO1TDOILUEE5BAAEEXAWRK2R3MRU
True labels       : WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
Accuracy: 80.0%

Result for subject 2
Constructed labels: MERMIMOOMUHJPXJOHUPKDORHL3GLOO7DUFDKFFTWEOOALZOP9R1CGZEY1Y19EWX65QUYU7NAK_4ACDDVDNAQXODBEV2B5EFDIDNR
True labels       : MERMIROOMUHJPXJOHUVLEORZP3GLOO7AUFDKEFTWEOOALZOP9ROCGZET1Y19EWX65QUYU7NAK_4YCJDVDNGQXODBEV2B5EFDIDNR
Accuracy: 87.0%

Overal accuracy: 83.5%
